<a href="https://colab.research.google.com/github/cabamarcos/SuperMask/blob/main/Prueba_pesos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import tensorflow as tf
from tensorflow import keras

from sklearn.model_selection import train_test_split

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler,ReduceLROnPlateau
from tensorflow.keras.datasets import mnist

import os

from keras.models import Sequential
from keras.layers import Dense


In [3]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")


11490434/11490434 [==============================] - 0s 0us/step
X_train shape: (60000, 28, 28)
y_train shape: (60000,)
X_test shape: (10000, 28, 28)
y_test shape: (10000,)


In [4]:
datagen = ImageDataGenerator(rotation_range=10,  zoom_range = 0.1,  width_shift_range=0.1,  height_shift_range=0.1)

In [5]:
# Learning rate annealer

reduce_lr = ReduceLROnPlateau(monitor='val_accuracy',
                                patience=3,
                                verbose=1,
                                factor=0.2,
                                min_lr=1e-6)

In [6]:
model = keras.models.Sequential([
    keras.layers.Conv2D(32, kernel_size=3, activation='relu', padding='same', input_shape=[28, 28, 1]),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(64, kernel_size=3, activation='relu', padding='same'),
    keras.layers.MaxPool2D(),
    keras.layers.Conv2D(32, kernel_size=3, activation='relu', padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(64, kernel_size=3, activation='relu', padding='same'),
    keras.layers.MaxPool2D(),
    keras.layers.Flatten(),
    keras.layers.Dropout(0.25),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(10, activation='softmax')
])

mask = keras.models.Sequential([
    keras.layers.Conv2D(32, kernel_size=3, activation='relu', padding='same', input_shape=[28, 28, 1]),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(64, kernel_size=3, activation='relu', padding='same'),
    keras.layers.MaxPool2D(),
    keras.layers.Conv2D(32, kernel_size=3, activation='relu', padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(64, kernel_size=3, activation='relu', padding='same'),
    keras.layers.MaxPool2D(),
    keras.layers.Flatten(),
    keras.layers.Dropout(0.25),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(10, activation='softmax')
])


model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
mask.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [7]:
X_train = X_train.reshape((60000, 28, 28, 1))

In [8]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.08)
history = model.fit_generator(datagen.flow(X_train, y_train, batch_size=64),
                              epochs=1, validation_data=(X_valid, y_valid), callbacks=[reduce_lr])
print("Maximum Train Accuracy : {} Maximum Validation Accuracy : {}".format(max(history.history['accuracy']), max(history.history['val_accuracy'])))


<ipython-input-8-ac6acc4430d8>:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(datagen.flow(X_train, y_train, batch_size=64),


863/863 [==============================] - 27s 23ms/step - loss: 0.3734 - accuracy: 0.8815 - val_loss: 0.1000 - val_accuracy: 0.9710 - lr: 0.0010
Maximum Train Accuracy : 0.8815217614173889 Maximum Validation Accuracy : 0.9710416793823242


In [9]:
# Obtiene los pesos de cada capa
for i, layer in enumerate(model.layers):
    if layer.get_weights():  # Comprueba si la capa tiene pesos
        weights = layer.get_weights()
        print(f"Capa {i}:")
        print("Pesos:")
        print(weights)
    else:
        print(f"Capa {i}: Esta capa no tiene pesos.")

Capa 0:
Pesos:
[array([[[[-5.79706393e-02,  9.57290083e-02, -8.02089870e-02,
          -8.74403641e-02, -1.29413739e-01, -3.41165625e-02,
           4.72433828e-02, -1.01403512e-01, -7.47332871e-02,
           1.25342771e-01, -1.18301250e-01, -9.94058400e-02,
           6.72716722e-02, -5.12166740e-03, -1.30456224e-01,
          -1.41572475e-01, -1.03663635e-02,  1.09143145e-01,
           1.16490729e-01,  3.27358134e-02, -6.68154582e-02,
          -8.37429911e-02,  1.69968791e-03,  8.16093981e-02,
           8.25226400e-03, -1.02219908e-02,  1.05907626e-01,
           4.65193130e-02, -1.76046461e-01, -6.24232814e-02,
          -2.12145224e-02, -3.40838321e-02]],

        [[-1.02244437e-01,  3.51609811e-02,  8.79391730e-02,
           1.49043724e-02, -8.27520620e-03, -3.57745551e-02,
           1.09929796e-02,  1.13705387e-02,  5.40879257e-02,
          -1.01000957e-01, -7.01877102e-02, -9.81921926e-02,
           1.91020407e-02,  5.03975898e-02, -1.14155114e-01,
           1.17008192e

In [12]:
model_layers= model.layers
mask_layers = mask.layers

# Iterar sobre cada capa y copiar los pesos
for i in range(len(model_layers)):
    weights = model_layers[i].get_weights()  # Obtener solo los pesos
    if weights:  # Comprobar si la capa tiene pesos
        mask_layers[i].set_weights(weights)

# Verificar que los pesos se han copiado correctamente
for i, layer in enumerate(mask_layers):
    weights = layer.get_weights()  # Obtener solo los pesos
    if weights:  # Comprobar si la capa tiene pesos
        print(f"Capa {i + 1}:")
        print("Pesos:")
        print(weights)
    else:
        print(f"La capa {i + 1} no tiene pesos.")


Capa 1:
Pesos:
[array([[[[-5.79706393e-02,  9.57290083e-02, -8.02089870e-02,
          -8.74403641e-02, -1.29413739e-01, -3.41165625e-02,
           4.72433828e-02, -1.01403512e-01, -7.47332871e-02,
           1.25342771e-01, -1.18301250e-01, -9.94058400e-02,
           6.72716722e-02, -5.12166740e-03, -1.30456224e-01,
          -1.41572475e-01, -1.03663635e-02,  1.09143145e-01,
           1.16490729e-01,  3.27358134e-02, -6.68154582e-02,
          -8.37429911e-02,  1.69968791e-03,  8.16093981e-02,
           8.25226400e-03, -1.02219908e-02,  1.05907626e-01,
           4.65193130e-02, -1.76046461e-01, -6.24232814e-02,
          -2.12145224e-02, -3.40838321e-02]],

        [[-1.02244437e-01,  3.51609811e-02,  8.79391730e-02,
           1.49043724e-02, -8.27520620e-03, -3.57745551e-02,
           1.09929796e-02,  1.13705387e-02,  5.40879257e-02,
          -1.01000957e-01, -7.01877102e-02, -9.81921926e-02,
           1.91020407e-02,  5.03975898e-02, -1.14155114e-01,
           1.17008192e